In [127]:
##### Visual String Pot
# This program takes an image and finds the centre of an orange stripe and green dot. It then use the length of the orange stripe to scale the difference between the clusters to calculate the length.
# it uses FAISS kmeans clustering to find a cluster of the approximate size of the stripe and dot.
# the stripe and dot are then rotated onto the principle component of the stripe
# the user defines a reduction and enchancment factotor which reduces the numbe rof pixcel and enchances the colours when the images are imported.
# the uses then defines the window to focus clustering and the aprox dimensions of the striope and dot
# the results are then exported as a csv with index, fileName, timeStamp, measurement, dotMedian, stripeMedian, scale, dotLength,

In [128]:
#### In version 1.1
# image correction and scaling simplified
# plotting impoved
# hyperparmter entry streamlined
# post calibration, offset and scaling
# improved layout to add option for other clustering methods later

In [154]:
## define functions

def loadImages(directory, file_list, sample, reduction, enchanced, dim, ench_type):
    ## Function takes file list and  if samples --> loads sample,
    # otherwise loads all images in file list, reducing by the reduction factor and enchancing by the enchance factor.
    # If dim --> crops image otherwise entire image is loaded

    timeStamp = []  # create empty list for time stamps
    index = 0  # set index to zero, used as counter for assignment  to array

    if not sample:  # if sample does not exist create a list of numbers to load
        sample = range(len(file_list))  # make sample the entire file list

    if not dim:  # open an image and get the image dimensions if no cropping is supplied
        image = np.array(Image.open(directory + file_list[0]).reduce(int((reduction))))  # load image to array
        h, w, d = tuple(image.shape)  # get shape of image in file
        dim = [0, h, 0, w, ]  # assign shape to dim list so entire image is imported

    # Create empty NP array for pixcels. All pictures are loaded into an array height, width, # images, colours
    pixcels = np.empty((dim[1] - dim[0],  # number of rows
                        dim[3] - dim[2],  # number of columns
                        len(sample),  # number of images
                        3),  # number of colour arrays per image + 3 for x and y and l
                       dtype='float32')

    for sam in sample:  # loop over each file in file_list
        clear_output(wait=True)  # clear output
        file = file_list[sam]  # set file as the sam'th entry in file list
        file_path = directory + file  # define file path
        ts = Image.open(file_path)._getexif()[36867]  # import timestamp as tS

        if enchanced:  # if an enchancments is call
            #open image and create enchancer
            if ench_type == 'Bright':
                enchancer = ImageEnhance.Brightness(Image.open(file_path).reduce(int((reduction))))
                image = np.array(enchancer.enhance(enchanced))  # load image and convert to array

            else:
                enchancer = ImageEnhance.Contrast(Image.open(file_path).reduce(int((reduction))))
                image = np.array(enchancer.enhance(enchanced))  # load image and convert to array

        else:  # if no enchancement is called
            image = np.array(Image.open(file_path).reduce(int((reduction))))  # load image into array
        # crop array and load into pixcels array
        pixcels[:, :, index, :] = image[dim[0]: dim[1],
                                  dim[2]: dim[3],
                                  :]

        timeStamp.append(ts)  # append time stamp to list
        index += 1
        print("importing " + file + " file # " + str(sam + 1) + " of " + str(len(file_list) + 1))
        print(ts)
    return (timeStamp, pixcels)

def determine_enchancement(in_dir, file_list, sample, reduction, enchancement, dim, ench_type, target_length):
    avgLength = 0
    while avgLength < target_length:
        timeStamp, pixcels = loadImages(in_dir, file_list, sample, reduction, enchancement, dim, ench_type)
        avgLength = np.reshape(pixcels, (-1,3))
        lengths = []
        for i in range(0,avgLength.shape[0]):
            lengths.append(np.linalg.norm(avgLength[i]))
        avgLength = sum(lengths)/len(lengths)

        if avgLength < target_length:
            enchancement += 1

    return enchancement

def clus2Image(clussArr, centers, recolour_dict, scale):
    ## defien function to take an array of cluster labels and centers and return an image array, with specified clusters recoulured if required
    if scale:
        Scaler = MinMaxScaler(feature_range=(0,255)) # define minmax scaler to scale colours to 0 --> 255
        centers = Scaler.fit_transform(centers)

    if recolour_dict: # if recolour provided
        for key in recolour_dict.keys(): # iterate over each key value pair
            centers[key] = np.asarray(recolour_dict[key]) # change the value of that center to the specified colour

    imgArr = centers[clussArr]

    return(imgArr)

def plotSamples(imgArray, ticks, title, titleData, scaleColours):
    ## function takes either an image array or array of cluster labels. checks if colours are valid and if required changes colour of selected clusters.
    # Then plots images over 2 columns and as many rows as required. user selects if ticks and headings --> supply heading data in list form

    if scaleColours: # if scaling colours is required
        Scaler = MinMaxScaler(feature_range=(0,255)) # define minmax scaller to scale colours to 0 --> 255
        imgArray = Scaler.fit_transform(imgArray) # scale image array RGB channels


    sampleSize = imgArray.shape[2]  # return the 3rd dimension of the array as the number of images to plot
    width = 2  # set number of columns to plot
    height = int(sampleSize / width)  # determine how many rows to plot
    index = 0  # set a counter to 0
    plt.close()  # close previous plot
    f, axarr = plt.subplots(height, width)  # create enough rows to plot all samples
    for row in range(0, height):  # iterate over each image row
        for h in range(0, height):  # iterare over each column
            axarr[h, row].imshow(imgArray[:, :, index, :].astype('uint8'))  # open image and plot
            if title:
                axarr[h, row].title.set_text(titleData[row])

            if not ticks:
                axarr[h, row].set_xticks([])
                axarr[h, row].set_yticks([])

            index += 1  # increase counter by 1

def faissCluster(pixcels, frac, startClus, noIt, reduction):
    ## define function to perfrom kmeans clustering using faiss
    no_clusters = startClus  # set inital number of clusters
    no_iterations = noIt  # set the no of iterations for each step of kmeans
    clusterMin = 1000  # set cluster min to enter while loop
    restart = 0
    pixcels = np.reshape(pixcels, (h*w*l,-1)) # reshape pixcels to a list of features
    d = pixcels.shape[1] # record the length of each feature to for faiss clustering definition

    if pixcels.dtype != 'float32': # check pixcel data type is suitable for faiss
        pixcels = pixcels.astype('float32') # if required change data type
        print("input array data type changed to "+str(pixcels.dtype)) # print conformation that data type change

    while clusterMin > frac:  # continue to increase number of clusters until the smallest cluster becomes sufficently small to just be the stripe
        #clear_output(wait=True)  # clear output
        print('clustering with ' + str(no_clusters) + "\n")
        kmeans = faiss.Kmeans(d, no_clusters, niter=no_iterations, verbose=True)  # define faiss kmeans object
        kmeans.train(pixcels)  # train kmeans object
        D, I = kmeans.index.search(pixcels, 1)  # return kmeans
        pixInClus = np.unique(I, return_counts=True)  # get counts # pixels in each cluster
        #colour = np.where(pixInClus[1] == min(pixInClus[1])) # assigns the smallest cluster as the stripe colour
        colour = np.where(
            abs(pixInClus[1] / pixInClus[1].sum() - frac) == abs(pixInClus[1] / pixInClus[1].sum() - frac).min())
        clusterMin = min(pixInClus[1]) / (
                    h * w * l)  # Calcultes the fraction of the picture occupied the the stripeColour
        no_clusters += 1  # Increase number of clusters by 1

        if clusterMin < 0.5 * frac: # check if cluster min is signifigantly smaller than the expected fraction
            frac = 1.15 * frac # increase fraction so clustering stops slighly earlier to stop the last step being too large
            no_clusters = startClus # restart the number of clusters
            clusterMin = 1000 # restart cluster min to restart while loop

        print("Cluster Min was " + str(clusterMin) + "\n") # print what the cluster min was

    I = np.reshape(I, (h,w,l,1))

    return (I, kmeans, colour[0])

def get_colour_cosine(kmeans, target_colour): # defien function to inspect all cluster centers and determine which has the clossestr cosine distance to the target colour
    cosineDistance = [] # empty list to store cosine distances

    for i in range(0,kmeans.centroids.shape[0]): # iterate over each centroid
        cosineDistance.append(distance.cosine(kmeans.centroids[i,:], target_colour)) # append the cosine distance to target colour

    trgCluster = np.where(cosineDistance == min(cosineDistance)) # find the closest
    trgCluster = trgCluster[0] # return single row of array for easy handling

    return trgCluster

def norm_to_target(labels, imgArray, trgClus, trg): # define function to normalise each image so as the target clusters appear similar for each image
    noStripe = 0

    imgArray = np.reshape(imgArray,(-1,l,3), order='F') # reshape fortran style so as each photo is a list of vectors

    ## find stripe mean and the picture mean
    if trg:
        labels = np.reshape(labels,(-1,l), order='F') # reshape labels fortran style so as each image is a list of labels
        picMeans_orig = [] # create empy list to append each picture mean
        for i in range(0,l): # iterate over each picture
            if np.where(labels[:,i] == trgClus)[0].shape[0] > 10: # check image contains atleast 10 stripe pixcels
                picMean = list(imgArray[np.where(labels[:,i] == trgClus),i,:].mean(axis=0)[0]) # if 10 stripe pixcel is found  calculate the mean value of the pixcels that are in stripe for that image
                picMeans_orig.append(picMean) # append this mean to pic mean

            else: # if no stripe pixcels are found append the avergae of the previous photo
                picMeans_orig.append(picMean)
                print("used Last pic mean "+str(picMean)+" for image number "+str(i))
                noStripe +=1

        picMeans_orig = np.array(picMeans_orig) # make picMeans_orig an array
        stripe_mean = picMeans_orig.mean(axis=0) # calculate stripe means by finding the average of pic means
        meanModifer = np.subtract(np.transpose(np.reshape(np.tile(stripe_mean, l),(3,l), order='F')),picMeans_orig) # determine the mean modifier

        for i in range(0,l): # iterate over each image
            imgArray[:,i,:] = np.transpose(np.transpose(imgArray[:,i,:])+np.reshape(np.repeat(meanModifer[i,:], h*w, axis=0), (d,h*w))) # apply the mean modifier

    imgArray[np.where(imgArray < 0)] = 0 # find anywhere the the rgb has gone below 0 and replace with 0
    imgArray = np.reshape(imgArray,(h*w*l,d)) # reshape img array to list of pixcels
    pixcelSums = imgArray.sum(axis=1) # find the sum of each pixcel
    pixcelSums[np.where(pixcelSums == 0)] = 1 # where the pixcel sum is 0 replace with 1 to avoid dev by 0 error, it wont matter as 0/1 = 0
    pixcelSums = np.reshape(np.tile(pixcelSums,3), (h*w*l,d), order='F') # reshape to correct shape
    imgArray = np.divide(imgArray, pixcelSums) # normalise by dividing by sum
    imgArray = np.reshape(np.reshape(imgArray,(h*w,l,d)), (h,w,l,d), order='F') # reshape to hand back to

    return imgArray, noStripe

def cleanIQR(upper, lower, singleValues, inplace, verb):
    iqr = np.subtract(*np.percentile(singleValues, [upper, lower]))
    med = np.percentile(singleValues, 50)
    minus = med - iqr
    plus = med + iqr
    toKeep = (minus < singleValues) & (singleValues < plus)
    if verb:
        plt.close()
        f, axarr = plt.subplots(2)  # create enough rows to plot all samples
        axarr[0].boxplot(singleValues)
        axarr[1].boxplot(singleValues[(minus < singleValues) & (singleValues < plus)])
        print('The min projected y value before cleaning is ' + str(singleValues.min()))
        print('The max projected y value before cleaning is ' + str(singleValues.max()))
        print('The min projected y value after cleaning is ' +
              str(singleValues[(minus < singleValues) & (singleValues < plus)].min()))
        print('The max projected y value after cleaning is ' +
              str(singleValues[(minus < singleValues) & (singleValues < plus)].max()))

    if inplace:
        return (singleValues[(minus < singleValues) & (singleValues < plus)])
    else:
        return (toKeep)

def find_rotation(coords):
    perpClean_min = 25
    perpClean_max = 75
    rotClean_min = 25
    rotClean_max = 75
    n=0
    while n < 2:
        pca = PCA(2) #define pca
        pca.fit(coords[0:2,:].T) # fit pca to the stripe cordinates
        Vh = pca.components_ # find rotation vector
        stripe_perp = coords[0:2,:].T @ Vh[:,1] # rotate stripe onto second principle component of stripe to clean
        stripe_perp = cleanIQR(perpClean_max,perpClean_min,stripe_perp, False, False) # remove outliers from stripe rot
        stripe_rot = coords[0:2,:].T @ Vh[:,0] # rotate stripe onto first principle component
        stripe_rot = stripe_rot[stripe_perp]
        coords = coords[:, stripe_perp]
        stripe_rot = cleanIQR(rotClean_max,rotClean_min,stripe_rot, False, False) # clean IQR
        coords = coords[:,stripe_rot]
        n += 1

    print("Stripe is roated "+str(math.acos(Vh[0,0]))+" degrees")
    return(Vh, coords)

def cluster_length(coordinates, Vh):
    prob_list = []
    length_list = []
    fail = []
    for i in range(0,l):
        try:
            pic = coordinates[0:2,np.where(coordinates[2,:] == i)[0]] # select from single photo
            pic = np.transpose(np.matmul(np.transpose(pic), Vh)) # transpose and mulitiply by rotation matrix
            pic = abs(np.rint(pic[0,:]).astype('int')) # round and multiply by negative 1
            count = np.bincount(pic) # count how many occourences for each x value
            max = count.max() # assume the highest count is actually the full stripe
            prob = count / max # calculate the probability of each x observation actually being a stripe
            start = np.where(prob > 0.75)[0].min() # take the start of the stripe as the first cordinate that is 75% likely to be a stripe
            finish = np.where(prob > 0.75)[0].max() # take the finish point of the stripe as the last cordinate that is atleast 75% likely to be a part of the stripe
            length = finish - start
            stripeProb = prob[start:finish].sum()/len(prob[start:finish])
            prob_list.append(stripeProb)
            length_list.append(length)

        except:
            fail.append(i)

    percentileLimit = np.nanpercentile(np.array(prob_list), 95)
    goodSample = np.where(np.array(prob_list) > percentileLimit)
    lengths = np.asarray(length_list)
    stripeLength = np.median(lengths[goodSample])


    return stripeLength

def addInd(w, h, l, pixcels, prince, Vh):
    pixcels = np.reshape(pixcels, (h*w*l,d))
    ## define function to add x and y cordinated, projected onto second principle component of the stripe to the pixcels array so as geometric position has an effect
    i, j, p = np.meshgrid(range(w), range(h), range(l))  # use mesh grid to create arrays of indices
    i = np.reshape(i, (w * h * l, 1)).astype('float32')  # reshape x ind to match reshaped pixcels
    j = np.reshape(j, (w * h * l, 1)).astype('float32')  # reshape y ind to suit reshaped pixcels
    searchArray = np.hstack((j, i))  # stack ontop
    del i
    del j
    del p
    if prince:  # only if using Vh
        searchArray = searchArray @ Vh[:, 1]  # multiply by rotation vector
        plt.close()
        plt.boxplot(searchArray)  # plot to check range
        searchArray = np.interp(searchArray, (searchArray.min(), searchArray.max()), (-1, 1))
        print('scaled min of dot_search is ' + str(searchArray.min()))
        print('scaled max of dot_search is ' + str(searchArray.max()))

    else:  # if no rotation matrix is specified note - this carries both x and y through. if wanting to drop x simply defien a straight rotation matrix
        plt.close()
        plt.boxplot(searchArray)  # plot to check range
        searchArray = np.interp(searchArray, (searchArray.min(), searchArray.max()), (-1, 1))
        print('scaled min of dot_search is ' + str(searchArray.min()))
        print('scaled max of dot_search is ' + str(searchArray.max()))
    searchArray = np.reshape(searchArray, (h * w * l, 1))
    searchArray = np.hstack((pixcels, searchArray))
    ## where is the y component dropped if not using Vh make it carry through
    return (searchArray.astype('float32'))

def GetStats(stripe_loc, perpClean_min, perpClean_max, rotClean_min, rotClean_max):
    pca = PCA(2)  # define scikitLearn PCA function
    pca.fit(stripe_loc[0:2, :].T)  # perform PCA on the x and y componets of the stripe loctions
    Vh = pca.components_  # extract the rotation matrix
    stripe_perp = stripe_loc[0:2, :].T @ Vh[:, 1]  # rotate stripe onto second principle component of stripe to clean
    stripe_perp = cleanIQR(perpClean_max, perpClean_min, stripe_perp, False, False)  # remove outliers from stripe rot
    stripe_rot = stripe_loc[0:2, :].T @ Vh[:, 0]  # rotate stripe onto first principle component
    stripe_rot = stripe_rot[stripe_perp]  # remove values which are removed in the perp cleaning process
    stripe_rot = cleanIQR(rotClean_max, rotClean_min, stripe_rot, True, False)
    ## do i need to shape stripe_perp? and stripe Rot to subset stripe Loc
    stripe_loc = stripe_loc[stripe_perp]  # remove perp outliers from stripe loc
    stripe_loc = stripe_loc[stripe_rot]  # remove rot outliers from loc
    stripeMedian = np.median(stripe_rot)  # find median of the stripe
    stripe_len = (
        abs(abs(np.min(stripe_rot)) - abs(np.max(stripe_rot))))  # finds the abs range of the stripe (known length)

    return (stripeMedian, stripe_len, Vh, stripe_loc)


In [130]:
## Import packages and set up environemnt
from os import listdir
from os.path import isfile, join
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from IPython.display import clear_output
import time
import faiss
from sklearn.decomposition import PCA
import vSp_functions_V1_1 as V
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
import math
import seaborn as sns

In [131]:
## define working directory and which files to sample
in_dir = '/mnt/veeringDL_storage/test_9/' # define directory containing all gopro images
out_dir = '/mnt/home/9.0 Data Jobs/' # define output directory
file_list = [f for f in listdir(in_dir) if isfile(join(in_dir, f))] # inspect directory and return list of files
print(str(len(file_list))+' Images found in folder')

10272 Images found in folder


In [132]:
## user defines which images to sample, trget size, enchancemt factor and type
sample = [10,4000,7000,10000] # Set which 4 images to sample
targetSize = 1200000 # define target size in pixcels
enchancement = 1 # define enchancement value
ench_type = 'Bright' # define enchancement type

In [133]:
## load sample images and plot
timeStamp, pixcels = loadImages(in_dir, file_list, [0], 1, False, False, False) # load a single image at full resolution
h, w, l, d = orig_shape = tuple(pixcels.shape) # return the dimensions of the original image
reduction = int((h*w)/targetSize) # caclulate the required reduction
print('A reduction factor of '+str(reduction)+' was adopted')
timeStamp, pixcels = loadImages(in_dir, file_list, sample, reduction, enchancement, False, ench_type)
plotSamples(pixcels, True, False, False, False)

importing G0023199.JPG file # 10001 of 10273
2022:04:23 14:58:44


<IPython.core.display.Javascript object>

In [134]:
## User defines where to crop image
h1 = 105 # horizontal (x) to start image at
h2 = h1+220 # horizontal (x) to end image at
v1 = 130 # vertical (y) to start image at
v2 = v1+90 # vertical (y) to end image at
dim = [v1, v2, h1, h2] # make dim variable as list to pass to load images
enchancement = 1 # define enchancement value
ench_type = 'Bright' # define enchancement type
enchancement_value = determine_enchancement(in_dir, file_list, sample, reduction, enchancement, dim, ench_type, 200)
timeStamp, pixcels = loadImages(in_dir, file_list, sample, reduction, enchancement_value, dim, ench_type)
print("Brightness used "+str(enchancement_value))
plotSamples(pixcels, True, False, False, False)

importing G0023199.JPG file # 10001 of 10273
2022:04:23 14:58:44
Brightness used 2


<IPython.core.display.Javascript object>

In [135]:
stripe_left = 117 # left bound of stripe
stripe_right = 201 # right bound of stripe
stripe_top = 57 # top bound of stripe
stripe_bottom = 75 # bottom bound of stripe
dot_left = 47 # left bound dot
dot_right =71 # right bound of dot
dot_top =19 # top bound of dot
dot_bottom = 40 # bottom bound of dot
area_factor = 1.1 # factor to multiply aproximmated area by
stepAfterStripeCluss = 1
stripe_area = abs(stripe_right - stripe_left) * abs(stripe_bottom - stripe_top) # calcluate area of stripe in pixcels
dot_area = abs(dot_right - dot_left) * abs(dot_bottom - dot_top) # calculate area of dot in pixcels
fig_area = abs(v2-v1) * abs(h2-h1) # calculate area of figure with
stripeFrac = area_factor * stripe_area / fig_area # expected fraction of image to be the stripe
dotFrac = area_factor * dot_area / fig_area # expected fraction of image to be the dot

print("The stripe fraction used is "+str(stripeFrac))
print("the dot fraction used is "+str(dotFrac))

The stripe fraction used is 0.084
the dot fraction used is 0.028000000000000004


In [136]:
## define the number of iterations and # clusters to start with for faiss Kmeans
it = 100 # # interations to perform in clustering
stripe_startNo = 1 # number of initial clusters

In [137]:
pixcels = allImages ## HANDY

In [11]:
## load all images, create image array to save RGB array before scaling and save original input array dimensions to rebuild later
timeStamp, pixcels = loadImages(in_dir, file_list, False, reduction, enchancement_value, dim, ench_type)
allImages = pixcels # save original RGB pixcels for plotting latter

importing G0023550.JPG file # 10272 of 10273
2022:04:23 15:04:35


In [138]:
h, w, l, d = orig_shape = tuple(pixcels.shape) # get shape of pixcels array
## Run faiss kmeans to find initial stripe cluster
features, noStripe = norm_to_target(False,pixcels,False,False)

In [139]:
plotSamples(features[:,:,sample,:]*255, False, False, False, False)

<IPython.core.display.Javascript object>

In [140]:
I, kmeans, stripeColour =  faissCluster(features, stripeFrac, stripe_startNo, it, False) # perform faiss clustering looking for stripe

clustering with 1


Sampling a subset of 256 / 203385600 for training
Clustering 256 points in 3D to 1 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.73 s
Cluster Min was 1.01 s, search 0.01 s): objective=3.54084 imbalance=1.000 nsplit=0       

clustering with 2


Sampling a subset of 512 / 203385600 for training
Clustering 512 points in 3D to 2 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 0.07876561565813903 s): objective=2.84806 imbalance=1.800 nsplit=0       



In [141]:
## use cosine distance to find cluster closest to orange
stripeColour = get_colour_cosine(kmeans, [255,169,0])
imgArr = clus2Image(I, kmeans.centroids[:,0:3], {stripeColour[0]: [0,1,0]}, False) # check array shapes here??
imgArr = np.reshape(imgArr, orig_shape)
plotSamples(imgArr[:,:,sample,:]*255, False, False, False, False)

<IPython.core.display.Javascript object>

In [142]:
## use know stripe locations to normalise colour accross data set using mean translate and then rgb chormisation
features, noStripe = norm_to_target(I,pixcels,stripeColour, True)
plotSamples(features[:,:,sample,:]*255, True, False, False, False)

used Last pic mean [80.0, 40.0, 32.0] for image number 267
used Last pic mean [166.0, 74.0, 46.0] for image number 683
used Last pic mean [150.0, 66.0, 38.0] for image number 1015
used Last pic mean [114.0, 62.0, 42.0] for image number 1303
used Last pic mean [90.0, 48.0, 34.0] for image number 1426
used Last pic mean [112.0, 50.0, 30.0] for image number 1454
used Last pic mean [130.0, 54.0, 36.0] for image number 1468
used Last pic mean [102.0, 52.0, 34.0] for image number 1533
used Last pic mean [192.0, 102.0, 70.0] for image number 1662
used Last pic mean [252.0, 120.0, 76.0] for image number 1937
used Last pic mean [156.0, 82.0, 58.0] for image number 1943
used Last pic mean [86.0, 44.0, 30.0] for image number 2108
used Last pic mean [244.0, 100.0, 60.0] for image number 2343
used Last pic mean [198.0, 90.0, 62.0] for image number 2563
used Last pic mean [146.0, 76.0, 52.0] for image number 2966
used Last pic mean [232.0, 104.0, 64.0] for image number 2977
used Last pic mean [186.0

<IPython.core.display.Javascript object>

In [143]:
## recluster using improved features
I, kmeans, stripeColour =  faissCluster(features, stripeFrac, 1, it, noStripe/l) # rerun kmeans on normalised pixcels

clustering with 1


Sampling a subset of 256 / 203385600 for training
Clustering 256 points in 3D to 1 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 1.01 s, search 0.01 s): objective=6.09924 imbalance=1.000 nsplit=0       

clustering with 2


Sampling a subset of 512 / 203385600 for training
Clustering 512 points in 3D to 2 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 0.08020496534661255 s): objective=7.94757 imbalance=1.793 nsplit=0       



In [144]:
## check the cluster found as the stripe is closest in colour to what was expected
stripeColour_cosine = get_colour_cosine(kmeans, [255,169,0]) # find stripe colour using cosine distance
if stripeColour_cosine != stripeColour:
    print("Error, the cluster found for stripes is inconsitent")

In [145]:
## plot final stripe clusters
imgArr = clus2Image(I, kmeans.centroids[:,0:3], {stripeColour[0]: [0,255,0]}, False)
imgArr = np.reshape(imgArr, orig_shape)
plotSamples(imgArr[:,:,sample,:], False, False, False, False)

<IPython.core.display.Javascript object>

In [146]:
## use PCA to find the long edge of the stripe and project coordinates onto this axis
stripe_coords = np.array(np.where(np.reshape(I, (h,w,l)) == stripeColour)) # get stripe locations from reshaped labels array
Vh, stripe_coords_clean = find_rotation(stripe_coords)
stripeLength = cluster_length(stripe_coords_clean, Vh)

Stripe is roated 1.6121191421279781 degrees


/tmp/ipykernel_116640/1212567542.py:257: RuntimeWarning: invalid value encountered in double_scalars
  stripeProb = prob[start:finish].sum()/len(prob[start:finish])


In [149]:
stripeLength

68.0

In [150]:
## now recluster looking for dots
I, kmeans, dotColour =  faissCluster(features, dotFrac, stripe_startNo, it, noStripe/l)

clustering with 1


Sampling a subset of 256 / 203385600 for training
Clustering 256 points in 3D to 1 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.74 s
Cluster Min was 1.01 s, search 0.01 s): objective=6.09924 imbalance=1.000 nsplit=0       

clustering with 2


Sampling a subset of 512 / 203385600 for training
Clustering 512 points in 3D to 2 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 0.08020496534661255 s): objective=7.94757 imbalance=1.793 nsplit=0       

clustering with 3


Sampling a subset of 768 / 203385600 for training
Clustering 768 points in 3D to 3 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 0.07920191989993391 s): objective=7.2761 imbalance=2.093 nsplit=0       

clustering with 4


Sampling a subset of 1024 / 203385600 for training
Clustering 1024 points in 3D to 4 clusters, redo 1 times, 100 iterations
  Preprocessing in 3.69 s
Cluster Min was 0.028552739230309324s): objective=6.62

In [151]:
## check the cluster found as dot colour is the closest to the expected colour
dotColour_cosine = get_colour_cosine(kmeans, [0,256,0]) # find stripe colour using cosine distance
if dotColour_cosine != dotColour:
    print("Error, the cluster found for dots is inconsitent")

In [152]:
imgArr = clus2Image(I, kmeans.centroids[:,0:3], {dotColour[0]: [0,255,0]}, False)
imgArr = np.reshape(imgArr, orig_shape)
plotSamples(imgArr[:,:,sample,:], False, False, False, False)

<IPython.core.display.Javascript object>

In [155]:
dot_coords = np.array(np.where(np.reshape(I, (h,w,l)) == dotColour)) # get stripe locations from reshaped labels array
Vdot, dot_coords_clean = find_rotation(dot_coords)
dotLength = cluster_length(dot_coords_clean , Vh)

Stripe is roated 1.6292744448379912 degrees


/tmp/ipykernel_116640/3130759675.py:256: RuntimeWarning: invalid value encountered in double_scalars
  stripeProb = prob[start:finish].sum()/len(prob[start:finish])
/home/veering/anaconda3/envs/visualStringpot/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/veering/anaconda3/envs/visualStringpot/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [156]:
dotLength

nan

In [ ]:
## define known geometry
knownLength = -87 # known length of the stripe, negative if measure should get larger when dot is closer to stripe
offset = -127.5 # known zero offset

/tmp/ipykernel_116640/1915765086.py:17: RuntimeWarning: invalid value encountered in double_scalars
  stripeProb = prob[start:finish].sum()/len(prob[start:finish])


Stripe is roated 1.6292744448379912 degrees


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:


stripeMedian = np.median(stripe_rot) # find median of the stripe
stripe_len = (abs(abs(np.min(stripe_rot)) - abs(np.max(stripe_rot)))) # finds the abs range of the stripe (known length)

In [57]:

## plot sample images with the principle componets on it for assesment
stripe_perp = stripe_loc[0:2,:].T @ Vh[:,1] # rotate stripe onto second principle component of stripe to clean
stripe_perp = cleanIQR(perpClean_max,perpClean_min,stripe_perp, True, False) # remove outliers from stripe rot



In [58]:

stripe_loc = np.array(
    np.where(np.reshape(I, (h, w, l)) == stripeColour))  # get stripe locations from reshaped labels array

perpClean_min = 25
perpClean_max = 75
rotClean_min = 26
rotClean_max = 75
stripe_perp = stripe_loc[0:2, :].T @ Vh[:, 1]  # rotate stripe onto second principle component of stripe to clean
stripe_perp = cleanIQR(perpClean_max, perpClean_min, stripe_perp, False, False)  # remove outliers from stripe rot
stripe_perp = np.vstack((stripe_perp, stripe_perp, stripe_perp))
stripe_loc = np.reshape(stripe_loc[stripe_perp], (3, -1))
stripe_rot = stripe_loc[0:2, :].T @ Vh[:, 0]  # rotate stripe onto first principle component
stripe_rot = cleanIQR(rotClean_max, rotClean_min, stripe_rot, False, False)
stripe_rot = np.vstack((stripe_rot, stripe_rot, stripe_rot))
stripe_loc = np.reshape(stripe_loc[stripe_rot], (3, -1))

pca = PCA(2)
pca.fit(stripe_loc[0:2, :].T)
Vh = pca.components_
stripe_rot = stripe_loc[0:2, :].T @ Vh[:, 0]  # rotate stripe onto first principle component
stripeMedian = np.median(stripe_rot)  # find median of the stripe
stripe_len = (
    abs(abs(np.min(stripe_rot)) - abs(np.max(stripe_rot))))  # finds the abs range of the stripe (known length)

In [59]:

## plot sample images with the principle componets on it for assesment
stripe_perp = stripe_loc[0:2, :].T @ Vh[:, 1]  # rotate stripe onto second principle component of stripe to clean
stripe_perp = cleanIQR(perpClean_max, perpClean_min, stripe_perp, True, False)  # remove outliers from stripe rot

stripePlot = [stripe_rot.min(),stripe_rot.max(), np.median(stripe_perp), stripe_perp.min(),stripe_perp.max(), stripeMedian]
plt.close()
check = clus2Image(I, kmeans.centroids, {int(0): [0,255,0]}, True) # get image array with stripe clusters changed to green
check = np.reshape(check, orig_shape) # reshape for plotting
check = check[:,:,1,:] # reduce to just sample images
check = Image.fromarray(check.astype('uint8'))

angle = 90 - math.degrees(math.acos(Vh[1,1]))
check = np.asarray(check.rotate(angle))
plt.imshow(check[:, :, :].astype('uint8'))
plt.plot([np.array(([stripePlot[0],0]) @ np.linalg.inv(Vh)[1])*1,
          np.array([stripePlot[1],0]) @ np.linalg.inv(Vh)[1]*1],
         [np.array([0,stripePlot[2]]) @ np.linalg.inv(Vh)[0]*1,
          np.array([0,stripePlot[2]]) @ np.linalg.inv(Vh)[0]*1], linewidth = 3)

plt.plot([np.array([stripePlot[5],0]) @ np.linalg.inv(Vh)[1]*1,
          np.array([stripePlot[5],0]) @ np.linalg.inv(Vh)[1]*1],
         [np.array([0,stripePlot[3]]) @ np.linalg.inv(Vh)[0]*1,
          np.array([0,stripePlot[4]]) @ np.linalg.inv(Vh)[0]*1], linewidth = 3)

## Delete variables not required for dot search to free up ram

in recolour


<IPython.core.display.Javascript object>

In [60]:
dot_startNo = len(kmeans.centroids) + stepAfterStripeCluss  # # of clusters to start finding the dot cluster
scale = knownLength / stripe_len  # calculate the pixcel to mm scale

In [262]:

## add the positions projected onto the 2nd principal component to aid in finding dot cluster
#features = np.reshape(features, (w*h*l,d))
dot_search = addInd(w, h, l, imgArray, True, Vh)  # add indices

<IPython.core.display.Javascript object>

scaled min of dot_search is -1.0
scaled max of dot_search is 1.0


In [263]:
scaler = StandardScaler()
dot_search = scaler.fit_transform(dot_search)
## Cluster to find dots

In [269]:
dot_search

array([[ 1.5131138e-01, -5.8497362e-02, -1.3853861e-01,  1.3716978e+00],
       [ 3.8065818e-01, -1.3827188e-01, -3.7719041e-01,  1.3716978e+00],
       [ 3.7192723e-01, -2.0333312e-01, -3.2018694e-01,  1.3716978e+00],
       ...,
       [-3.9452139e-01,  1.7238465e-01,  4.0057316e-01, -1.3716978e+00],
       [-1.7625059e-01, -1.3058448e-03,  2.4224305e-01, -1.3716978e+00],
       [ 9.7931951e-01, -9.2149585e-02, -1.1791233e+00, -1.3716978e+00]],
      dtype=float32)

In [273]:
I, kmeans, dotColour = faissCluster(dot_search, dotFrac, 3, it)  # perform faiss clustering looking for dot

clustering with 3


Sampling a subset of 768 / 98049600 for training
Clustering 768 points in 4D to 3 clusters, redo 1 times, 10 iterations
  Preprocessing in 2.15 s
Cluster Min was 0.09695952864672575s): objective=1445.9 imbalance=1.291 nsplit=0       

clustering with 4


Sampling a subset of 1024 / 98049600 for training
Clustering 1024 points in 4D to 4 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.90 s
Cluster Min was 0.09289336213508265s): objective=1568.95 imbalance=1.152 nsplit=0       

clustering with 5


Sampling a subset of 1280 / 98049600 for training
Clustering 1280 points in 4D to 5 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.90 s
Cluster Min was 0.029145044956838173): objective=1621.9 imbalance=1.354 nsplit=0       



In [276]:
dotColour[0][0]

4

In [ ]:


imgArr = clus2Image(I, kmeans.centroids[:,0:3], {dotColour[0][0]: [255,125,0]}, False)
imgArr = np.reshape(imgArr, orig_shape)

In [282]:
plotSamples(imgArr[:,:,sample,:], False, False, False, False)

<IPython.core.display.Javascript object>

In [278]:
imgArray = norm_to_target(I,pixcels,dotColour[0][0])

used Last pic mean [174.0, 238.0, 128.0] for image number 12
used Last pic mean [90.0, 130.0, 74.0] for image number 285
used Last pic mean [68.0, 92.0, 50.0] for image number 335
used Last pic mean [56.0, 56.0, 54.0] for image number 1258
used Last pic mean [172.0, 192.0, 84.0] for image number 1419
used Last pic mean [120.0, 212.0, 84.0] for image number 1436
used Last pic mean [94.0, 188.0, 64.0] for image number 1454
used Last pic mean [110.0, 210.0, 76.0] for image number 1620
used Last pic mean [140.0, 234.0, 100.0] for image number 1748
used Last pic mean [122.0, 182.0, 86.0] for image number 1758
used Last pic mean [124.0, 250.0, 78.0] for image number 1766
used Last pic mean [206.0, 255.0, 132.0] for image number 1835
used Last pic mean [160.0, 255.0, 126.0] for image number 2173
used Last pic mean [160.0, 255.0, 126.0] for image number 2174
used Last pic mean [126.0, 166.0, 74.0] for image number 2200
used Last pic mean [46.0, 48.0, 44.0] for image number 2242
used Last pic m

In [281]:
plotSamples(imgArray[:,:,sample,:]*255, False, False, False, False)

<IPython.core.display.Javascript object>

In [322]:
dot_search = addInd(w, h, l, np.reshape(imgArray,(w*h*l,d)), True, Vh)  # add indices
scaler = StandardScaler()
dot_search = scaler.fit_transform(dot_search)
I, kmeans, dotColour = faissCluster(np.reshape(imgArray,(w*h*l,d)), dotFrac, 3, it)  # perform faiss clustering looking for dot

<IPython.core.display.Javascript object>

scaled min of dot_search is -1.0
scaled max of dot_search is 1.0
clustering with 3


Sampling a subset of 768 / 98049600 for training
Clustering 768 points in 3D to 3 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.85 s
Cluster Min was 0.04697909017476869s): objective=13.8095 imbalance=2.329 nsplit=0       

clustering with 4


Sampling a subset of 1024 / 98049600 for training
Clustering 1024 points in 3D to 4 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.81 s
Cluster Min was 0.040005191250142784): objective=13.6107 imbalance=2.088 nsplit=0       

clustering with 5


Sampling a subset of 1280 / 98049600 for training
Clustering 1280 points in 3D to 5 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.76 s
Cluster Min was 0.034033489172826815): objective=13.5026 imbalance=2.010 nsplit=0       

clustering with 6


Sampling a subset of 1536 / 98049600 for training
Clustering 1536 points in 3D to 6 clusters, redo 1 times, 10 iterations
  Preprocessing in 1.

In [326]:
dotColour[0]


array([3])

In [325]:
kmeans.centroids*255

array([[193.7503   ,  46.009144 ,  15.2404995],
       [ 84.98696  ,  86.68643  ,  83.32663  ],
       [ 94.44845  ,  58.843296 , 101.708275 ],
       [ 69.67049  , 146.70241  ,  38.627136 ],
       [ 89.83996  , 101.65454  ,  63.505573 ],
       [ 14.164419 ,   9.688826 , 231.14674  ]], dtype=float32)

In [345]:
imgArr = clus2Image(I, kmeans.centroids[:,0:3], {dotColour[0][0]: [0,255,0]}, False)
imgArr = np.reshape(imgArr, orig_shape)
plotSamples(imgArr[:,:,[1950,2950,3950,4950],:], False, False, False, False)

in recolour


<IPython.core.display.Javascript object>

In [317]:
colours.shape

(8, 3)

In [318]:
check = colours[labels]

In [320]:
check[0,:,:]

array([[-0.10568146,  0.3831842 , -0.16583098]], dtype=float32)

In [309]:
kmeans.centroids[:,0:3]

array([[-0.10568146,  0.3831842 , -0.16583098],
       [-0.14869453,  0.3067157 , -0.0721252 ],
       [-0.49998605,  1.9897408 , -0.9703579 ],
       [ 0.4755983 , -1.7018244 ,  0.81591755],
       [-2.2717893 , -2.627934  ,  3.947715  ],
       [ 2.7830863 , -1.1270809 , -1.6680276 ],
       [-0.92369854, -0.4543715 ,  1.046424  ],
       [-0.14077854,  0.16740128,  0.021208  ]], dtype=float32)

In [284]:

check = clus2Image(I, kmeans.centroids[:, 0:3],
                   {dotColour[0][0]: [0, 255, 0]}, True)  # get image array with dot clusters changed to green
check = np.reshape(check[:, 0:2], orig_shape)  # reshape for plotting
check = check[:, :, sample, :]  # reduce to just sample images
plotSamples(check, True, False, False, False)  # plot sample images to check clustering

in recolour


ValueError: cannot reshape array of size 2674080 into shape (90,220,4952,3)

In [299]:
check = check[:, :, sample, :]  # reduce to just sample images
plotSamples(check, True, False, False, False)

<IPython.core.display.Javascript object>

In [61]:
I = np.reshape(I, (h*w*l,1))
dot_loc = np.array(
    np.where(np.reshape(I, (h, w, l)) == dotColour))  # get dot locations x,y,l from reshaped labels array
dot_perp = dot_loc[0:2, :].T @ Vh[:, 1]  # rotate dot x and y onto second principle component of stripe to clean
perp_keep = cleanIQR(85, 15, dot_perp, False,
                     True)  # get true fals of values to keep in dot_loc based on outliers from dot on the 2nd principle component
## clean dot loc by perp keep
perp_keep = np.reshape(perp_keep, (1, -1))  # make 2d array with 1 row
perp_keep = np.vstack(
    (perp_keep, perp_keep, perp_keep))  # stack logical ontop of itself 3 times to match shape of dot_loc
dot_loc = np.reshape(dot_loc[perp_keep], (3, -1))  # subset dot_loc using perp_keep and return to origninal shape
dot_rot = dot_loc[0:2, :].T @ Vh[:, 0]  # rotate the remianing dost onto first principle component of stripe
rot_keep = cleanIQR(90, 10, dot_rot, False, True)  # clean dots by IQR
rot_keep = np.reshape(rot_keep, (1, -1))  # reshape to make 2d array with 1 row
rot_keep = np.vstack((rot_keep, rot_keep, rot_keep))  # stack to match shape of dot_loc
dot_loc = np.reshape(dot_loc[rot_keep], (3, -1))  # subset using rot_keep and reshape to original shape
dot_rot = dot_loc[0:2, :].T @ Vh[:, 0]  # rotate remaining dots onto first principal component
dot_rot = np.vstack((dot_rot, dot_loc[2, :]))  # stack the image number and rotated values ontop of eachother
dotsDF = pd.DataFrame(np.transpose(dot_rot),
                      columns=['Dot', 'Pic'])  # change fron NP array to data frame for easy grouping
knownLength = -87  # known length of the stripe, negative if measure should get larger when dot is closer to stripe
offset = -127.5  # known zero offset
scale = knownLength / stripe_len  # calculate the pixcel to mm scale
dataOut = dotsDF.groupby('Pic').median()
dataOut
dataOut = dataOut.merge(dotsDF.groupby('Pic').count(), left_index=True, right_index=True)
dataOut = dataOut.merge(pd.DataFrame(file_list), left_index=True, right_index=True)
dataOut = dataOut.merge(pd.DataFrame(timeStamp), left_index=True, right_index=True)
dataOut = dataOut.rename(columns={'Dot_x': 'Dot', 'Dot_y': 'Count', '0_x': 'file', '0_y': 'timeStamp'})
dataOut = dataOut[dataOut['Count'] > 0.6 * dot_area]  # change back to 0.6
dataOut = dataOut[dataOut['Count'] < 1.4 * dot_area]  #change back to 1.4
dataOut['measure'] = ((abs(stripeMedian - dataOut.Dot) * scale) - offset) * 1.17
dataOut = dataOut.sort_values('timeStamp')
dataOut.to_csv(out_dir + 'vSp_gp+220507.csv')
print("dataOut saved in " + out_dir + 'vSp.csv\n\n')
print(str(dataOut.head()) + '\n\n')
print(dataOut.describe())

plt.close()
plt.plot(dataOut.timeStamp, dataOut.measure)

dataOut = dotsDF.groupby('Pic').median()
dataOut = dataOut.merge(dotsDF.groupby('Pic').count(), left_index=True, right_index=True)
dataOut = dataOut.merge(pd.DataFrame(file_list), left_index=True, right_index=True)
dataOut = dataOut.merge(pd.DataFrame(timeStamp), left_index=True, right_index=True)
dataOut = dataOut.rename(columns={'Dot_x': 'Dot', 'Dot_y': 'Count', '0_x': 'file', '0_y': 'timeStamp'})
dataOut.Count.max()

<IPython.core.display.Javascript object>

The min projected y value before cleaning is -87.15477297382893
The max projected y value before cleaning is 11.366433304501744
The min projected y value after cleaning is -32.18590660059405
The max projected y value after cleaning is -3.1892310617578383


<IPython.core.display.Javascript object>

The min projected y value before cleaning is -220.93659893802646
The max projected y value before cleaning is -0.20760608775345651
The min projected y value after cleaning is -140.05829340213754
The max projected y value after cleaning is -1.2581598174300708
dataOut saved in /mnt/home/9.0 Data Jobs/vSp.csv


              Dot  Count          file            timeStamp   measure
1534.0 -53.733041    632  G0020005.JPG  2022:04:20 15:04:17  9.221846
5483.0 -53.681139    630  G0020006.JPG  2022:04:20 15:04:18  9.161148
2257.0 -53.940647    634  G0020007.JPG  2022:04:20 15:04:19  9.464638
521.0  -53.836844    632  G0020008.JPG  2022:04:20 15:04:20  9.343242
3053.0 -53.888745    624  G0020009.JPG  2022:04:20 15:04:21  9.403940


               Dot        Count      measure
count  7427.000000  7427.000000  7427.000000
mean    -78.585808   631.720075    38.286740
std      23.922561    41.612022    27.977033
min    -117.530456   387.000000     2.335800
25%    -115.066037   613.000000    19.97580

<IPython.core.display.Javascript object>

2264